In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from pymongo import MongoClient
from datetime import datetime

def _connect_mongo(host, port, username, password, db):
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
        
    return conn[db]

def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    db =_connect_mongo(db=db, host=host, port=port, username=username, password=password)
    cursor = db[collection].find(query)
        
    return list(cursor)

def _build_vector(event):
    # vector_len = 290
    # f_vector = np.zeros(vectror_len)
    
    m_country = assign_country_mapper()
    m_event = assign_event_type_mapper()
    m_agent_os = assign_agent_os_mapper()
    m_agent_name = assign_agent_name_mapper()
    
    f_vector_evty = np.zeros(4)
    f_vector_evty[m_event[event['type']]] = 1
    
    
    dt = datetime.utcfromtimestamp(event['timestamp'])
    minutes = (dt.hour * 60) + dt.minute
    day = dt.weekday()
    f_vector_dt = np.zeros(2)
    f_vector_dt[0] = round(minutes / 1439, 5)
    f_vector_dt[1] = round(day / 6, 5)
    
    latitude = event['geoip']['latitude'][0]
    longitude = event['geoip']['longitude'][0]
    f_vector_geo = np.zeros(2)
    f_vector_geo[0]= round((latitude + 90) / 180,5)
    f_vector_geo[1]= round((longitude + 180) / 360, 5)
    
    f_vector_agos = np.zeros(8)
    os = event['agent']['os'][0]
    if os in m_agent_os:
        f_vector_agos[m_agent_os[os]] = 1
    else:
        f_vector_agos[-1] = 1
        
    f_vector_agna = np.zeros(26)
    name = event['agent']['name'][0]
    if name in m_agent_name:
        f_vector_agna[m_agent_name[name]] = 1
    else:
        f_vector_agna[-1] = 1
        
    f_vector_country = np.zeros(248)
    country = event['geoip']['country'][0]
    if country in m_country:
        f_vector_country[m_country[country]] = 1
    else:
        f_vector_country[-1] = 1
    
    # without geo and without country features
    # return np.concatenate([f_vector_evty, f_vector_dt, f_vector_agos, f_vector_agna])

    # without country feature
    # return np.concatenate([f_vector_evty, f_vector_dt, f_vector_geo, f_vector_agos, f_vector_agna])
    
    # all features
    return np.concatenate([f_vector_evty, f_vector_dt, f_vector_geo, f_vector_agos, f_vector_agna, f_vector_country])

def vectorize(json):
    events = sorted(np.concatenate([json['connects'], json['plays'], json['h5liveStats'],json['closes']]), key=lambda d: d['timestamp'])
    vector_list = []
    if len(events) == 0:
        return None
    for event in events:
        vector_list.append(_build_vector(event))
    return np.stack(vector_list)

def add_event_types(json):
    if 'rtmpStats' in json:
        rtmpStats = json['rtmpStats']
        for x in rtmpStats:
            x['type']='rtmp'
    
    if 'connects' in json:
        connects = json['connects'] 
        for x in connects:
            x['type']='connect'
        
    if 'plays' in json:
        plays = json['plays']
        for x in plays:
            x['type']='play'
            
    if 'h5liveStats' in json:
        h5liveStats = json['h5liveStats']
        for x in h5liveStats:
            x['type']='h5live'
    
    if 'closes' in json:
        closes = json['closes']
        for x in closes:
            x['type']='close'  
    
    return json

In [3]:
import json

def assign_country_mapper():
    with open('../../src/util/country_mapper.json') as f:
        d = json.load(f)
        return d

def assign_event_type_mapper():
    with open('../../src/util/event_type_mapper.json') as f:
        d = json.load(f)
        return d
    
def assign_agent_os_mapper():
    with open('../../src/util/agent_os_mapper.json') as f:
        d = json.load(f)
        return d
    
def assign_agent_name_mapper():
    with open('../../src/util/agent_name_mapper.json') as f:
        d = json.load(f)
        return d

In [4]:
# load data from MongoDB
misuses = read_mongo('dataset1', 'misuse_data')
regulars = read_mongo('dataset1', 'regular_data')

In [5]:
v_misuses = []
v_regulars = []

for x in misuses:
    x = add_event_types(x)
    y = vectorize(x)
    if y is not None:
        # print(y.shape)
        v_misuses.append(y)

for x in regulars:
    x = add_event_types(x)
    y = vectorize(x)
    if y is not None:
        # print(y.shape)
        v_regulars.append(y)

In [6]:
train_size = (150,550)
im = torch.tensor([[1,0], [0,1]])

X_train_misuse = v_misuses[:train_size[0]:1]
X_train_regular = v_regulars[:train_size[1]:1]
X_train_final = X_train_misuse + X_train_regular

y_train = np.zeros_like(list(range(len(X_train_final))))

for i in range(len(y_train)):
    if i >= len(X_train_misuse):
        y_train[i] = 1

max_length_training = max([arr.shape[0] for arr in X_train_final])
padded_train_data = np.array([np.pad(arr,((0, max_length_training - arr.shape[0]), (0,0)), mode='constant') for arr in X_train_final])

tensor_train_data = Variable(torch.tensor(padded_train_data, dtype=torch.float64))
tensor_train_labels = Variable(im[torch.tensor(y_train)])


X_test_misuse = v_misuses[train_size[0]::1]
X_test_regular = v_regulars[train_size[1]::1]
X_test_final = X_test_misuse + X_test_regular

y_test = np.zeros_like(list(range(len(X_test_final))))

for i in range(len(y_test)):
    if i >= len(X_test_misuse):
        y_test[i] = 1

max_length_testing = max([arr.shape[0] for arr in X_test_final])
padded_test_data = np.array([np.pad(arr,((0, max_length_testing - arr.shape[0]), (0,0)), mode='constant') for arr in X_test_final])

tensor_test_data = Variable(torch.tensor(padded_test_data, dtype=torch.float64))
tensor_test_labels = Variable(im[torch.tensor(y_test)])

print('Training: ', tensor_train_data.shape, ' ', tensor_train_labels.shape)
print('Testing: ', tensor_test_data.shape, ' ', tensor_test_labels.shape)

Training:  torch.Size([400, 4000, 290])   torch.Size([400, 2])
Testing:  torch.Size([303, 3300, 290])   torch.Size([303, 2])


In [7]:
input_size = tensor_train_data.shape[2]
hidden_size = 64
num_layers = 1
num_classes = 2
num_epochs = 10
learning_rate = 0.003
batch_size = 16

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, seq_length):
        super(LSTMClassifier, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc3 = nn.Linear(hidden_size*2, num_classes)
        self.fc2 = nn.Linear(hidden_size*2, hidden_size*2)
        self.fc1 = nn.Linear(hidden_size, hidden_size*2)
        self.sigm = nn.Sigmoid()
        self.soft = nn.Softmax()

    def forward(self, x):
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        lstm_out, (hn, cn) = self.lstm(x, (h_0, c_0))
        
        # use last output of the last lstm layer
        out = lstm_out[:, -1, :]
    
        # first fully connected layer
        out = self.fc1(out)

        # second fully connected layer
        out = self.fc2(out)

        # third fully connected layer
        out = self.fc3(out)
        
        out = self.sigm(out)

        return out


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes, tensor_train_data.shape[0]).to(device)

# loss_func = torch.nn.MSELoss()
# loss_func = torch.nn.CrossEntropyLoss()
loss_func = torch.nn.BCELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


#define for filename
num_fclayers = 3
last_func = 'sigm'
l_func = 'BCE'

In [9]:
logging = True
filename = 'r' + num_layers + '_' +'f'+ num_fclayers + '_' +last_func+ '_' + 'h' + hidden_size + '_' + 'b' + batch_size + '_' + 'l' + learning_rate
filepath = "./logs/" +filename+".txt"
if logging:
    with open(filepath, 'w') as f:
        f.write('Start logging loss' + '\n')

# set train mode 
model.train()

# Training loop
total_step = len(tensor_train_data) // batch_size
for epoch in range(num_epochs):
    for i in range(0, len(tensor_train_data), batch_size):
        
        # Mini-batch data
        batch_inputs = tensor_train_data[i:i+batch_size, :, :].float()
        batch_labels = tensor_train_labels[i:i+batch_size]
        
        # Forward pass
        outputs = model.forward(batch_inputs)
        loss = loss_func(outputs.float(), batch_labels.float())
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Print statistics (adjust to modolu)
        if (i+1) % 1 == 0:
            message = 'Epoch [{}/{}], Batch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i//batch_size+1, total_step, loss.item())
            print(message)
            if logging:
                # log results
                with open(filepath, 'a') as f:
                    f.write(message+'\n')

Epoch [1/100], Batch [1/25], Loss: 0.6725
Epoch [1/100], Batch [2/25], Loss: 0.6463
Epoch [1/100], Batch [3/25], Loss: 0.6207
Epoch [1/100], Batch [4/25], Loss: 0.5951
Epoch [1/100], Batch [5/25], Loss: 0.5691
Epoch [1/100], Batch [6/25], Loss: 0.5424
Epoch [1/100], Batch [7/25], Loss: 0.8120
Epoch [1/100], Batch [8/25], Loss: 0.9360
Epoch [1/100], Batch [9/25], Loss: 0.9404
Epoch [1/100], Batch [10/25], Loss: 0.9329
Epoch [1/100], Batch [11/25], Loss: 0.9186
Epoch [1/100], Batch [12/25], Loss: 0.9006
Epoch [1/100], Batch [13/25], Loss: 0.8811
Epoch [1/100], Batch [14/25], Loss: 0.8613
Epoch [1/100], Batch [15/25], Loss: 0.8420
Epoch [1/100], Batch [16/25], Loss: 0.8234
Epoch [1/100], Batch [17/25], Loss: 0.8059
Epoch [1/100], Batch [18/25], Loss: 0.7894
Epoch [1/100], Batch [19/25], Loss: 0.7738
Epoch [1/100], Batch [20/25], Loss: 0.7591
Epoch [1/100], Batch [21/25], Loss: 0.7452
Epoch [1/100], Batch [22/25], Loss: 0.7319
Epoch [1/100], Batch [23/25], Loss: 0.7190
Epoch [1/100], Batch

In [10]:
# set evaluation mode
model.eval()

with torch.no_grad():
    outputs = model(tensor_test_data.float())

_, preds = torch.max(outputs, dim=1)
acc= (preds == tensor_test_labels.argmax(1)).sum().item() / tensor_test_labels.shape[0]


print("accuracy", acc)

accuracy 0.6006600660066007
